# Rede Neural Artificial para Identificação de Fake News



---

Declarando bibliotecas:

In [3]:
from google.colab import drive                        #lidar com o drive
import os                                             #pode vir a ser útil quando tivermos mais arquivos
import pandas as pd                                   #trabalhar com dataframes
import numpy as np                                    #Ajudar com a álgebra
!pip install googletrans==3.1.0a0                              #instalar a biblioteca deep_translator
from googletrans import Translator                    #traduzir t#exto
drive.mount('/content/drive', force_remount=True)     #fazer setup do drive
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from calendar import month_name

     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 1.0MB 6.3MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 71kB 5.9MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp36-none-any.whl size=16368 sha256=571d388daca66f2b78731515c00f9f77f40c00a52e21ca1cd55432dbeb6db6ad
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=fa7854af2536fa4d4e48bb5d4f3c1d8c447a5055e1bc9219b38751b3232f211a
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars
Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_

## Treinamento e teste inicial será realizado com banco de dados em inglês. Aqui importamos e processamos os arquivos:

In [ ]:
path = '//content//drive//MyDrive//Colab Notebooks//'
df_fake = pd.read_csv(path + 'Fake.csv',encoding = 'UTF-8')
df_true = pd.read_csv(path + 'True.csv',encoding = 'UTF-8

')

In [ ]:
df_fake['reality'] = 0
df_true['reality'] = 1
df_text = df_true.append(df_fake, ignore_index = True)
df_text['text'] = df_text['title'].str.lower() + ' ' + df_text['text'].str.lower()
df_text.drop(columns = ['subject','date','title'], inplace = True)

In [ ]:
for word in stopwords.words('english'):
  df_text['text'] = df_text['text'].str.replace(' ' + word + ' ','')

for word in month_name:
  df_text['text'] = df_text['text'].str.replace(' ' + word.lower() + ' ','')

#df_text_copy = df_text.copy()
#for i, row in df_text_copy.iterrows():
#  for char in row['text']:
#    if char.isdigit():
#      df_text['text'] = df_text['text'].str.replace(char,'')

df_text

,text,reality
0,"as u.s. budget fight looms, republicans flipfi...",1
1,u.s. militaryaccept transgender recruitsmonday...,1
2,senior u.s. republican senator: 'let mr. muell...,1
3,fbi russia probe helpedaustralian diplomat tip...,1
4,trump wants postal servicecharge 'much more'am...,1
...,...,...
44893,mcpain: john mccain furiousiran treated us sai...,0
44894,justice? yahoo settles e-mail privacy class-ac...,0
44895,sunnistan: usallied ‘safe zone’ plantake terri...,0
44896,howblow $700 million: al jazeera america final...,0


In [ ]:
y = np.array(df_text['reality'])  # é um array com os zeros e uns da coluna reality
X = np.array(df_text['text'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  # separando os arrays entre teste e treino

max = 1000
vectorizer = CountVectorizer(max_features = max)
X_train = vectorizer.fit_transform(X_train)  # cria matriz que mostra a relação que cada token aparece em cada texto
X_test = vectorizer.transform(X_test)  # 
print(vectorizer.get_feature_names())
X_train = preprocessing.normalize(X_train)  # coloca todos os valores da matriz na mesma ordem de grandeza
X_test = preprocessing.normalize(X_test)

['000', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '200', '2014', '2015', '2016', '2017', '21', '21st', '21wire', '22', '23', '24', '25', '26', '27', '28', '29', '30', '40', '50', '500', '60', 'abortion', 'about', 'absolutely', 'access', 'according', 'account', 'accused', 'across', 'act', 'action', 'actually', 'added', 'adding', 'administration', 'adviser', 'affairs', 'african', 'after', 'again', 'agencies', 'agency', 'agenda', 'agents', 'ago', 'agreement', 'ahead', 'aid', 'air', 'al', 'all', 'allies', 'almost', 'along', 'already', 'also', 'although', 'always', 'ambassador', 'amendment', 'america', 'american', 'americans', 'amid', 'among', 'an', 'and', 'andis', 'ands', 'announced', 'another', 'anti', 'anyone', 'anything', 'apparently', 'appeared', 'appears', 'arab', 'arabia', 'are', 'area', 'areas', 'armed', 'army', 'around', 'as', 'asked', 'assault', 'at', 'attack', 'attacks', 'attime', 'attorney', 'authorities', 'away', 'back', 'backed', 'bad', 'ban', 'b

## Com os dados processados, modelamos a rede neural e a treinamos:

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(30,activation='relu'))
model.add(keras.layers.Dense(20,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
fitModel = model.fit(X_train.toarray(), y_train, validation_split=0.33)

705/705 [==============================] - 2s 2ms/step - loss: 0.3125 - accuracy: 0.8784 - val_loss: 0.0260 - val_accuracy: 0.9935


## Em seguida, verificamos que a performance do modelo é satisfatório:

In [ ]:
prediction = model.evaluate(X_test,y_test)
print(prediction)

351/351 [==============================] - 1s 2ms/step - loss: 0.0271 - accuracy: 0.9923
[0.027062900364398956, 0.9923385381698608]


# Tratamento dos dados brasileiros tokenizados

Lendo o csv da base